# PCM Thermal Storage Module

Models phase change material (PCM) thermal storage for capturing excess thermal energy. Utilizes latent heat of fusion for high energy density storage at near-constant temperature.

## Energy Analysis

**Total thermal energy stored (sensible + latent):**
$$Q_{PCM} = m_{pcm}\left[C_{p,s}(T_m - T_i) + \Delta h_m + C_{p,l}(T_f - T_m)\right]$$

**State of charge based on enthalpy:**
$$SOC = \frac{h - h_{solid}}{h_{liquid} - h_{solid}}$$

**Energy balance:**
$$\frac{dE_{stored}}{dt} = \dot{Q}_{in} - \dot{Q}_{out} - \dot{Q}_{loss}$$

**Heat loss:**
$$\dot{Q}_{loss} = UA(T_{pcm} - T_{amb})$$

## Exergy Analysis

**Exergy content (sensible + latent):**
$$Ex_{PCM} = m_{pcm}C_{p,s}\left[(T_m - T_i) - T_0\ln\frac{T_m}{T_i}\right] + m_{pcm}\Delta h_m\left(1 - \frac{T_0}{T_m}\right) + m_{pcm}C_{p,l}\left[(T_f - T_m) - T_0\ln\frac{T_f}{T_m}\right]$$

**Simplified exergy at melting temperature:**
$$Ex_{stored} = E_{stored}\left(1 - \frac{T_0}{T_m}\right)$$

**Exergy destruction (charging):**
$$\dot{Ex}_{d,charge} = \dot{Q}_{charge}\left(1 - \frac{T_0}{T_{source}}\right) - \dot{Q}_{charge}\left(1 - \frac{T_0}{T_{pcm}}\right)$$

**Exergy loss:**
$$\dot{Ex}_{loss} = \dot{Q}_{loss}\left(1 - \frac{T_0}{T_{pcm}}\right)$$

## Parameters

In [1]:
# PCM Default Parameters (Paraffin wax)
const PCM_DEFAULTS = (
    ρ = 880.0,              # Density [kg/m³]
    C_p_s = 2.0e3,          # Specific heat solid [J/(kg·K)]
    C_p_l = 2.2e3,          # Specific heat liquid [J/(kg·K)]
    Δh_m = 200.0e3,         # Latent heat of fusion [J/kg]
    T_m = 323.15,           # Melting temperature [K] (50°C)
    k_s = 0.24,             # Thermal conductivity solid [W/(m·K)]
    k_l = 0.15,             # Thermal conductivity liquid [W/(m·K)]
    UA_loss = 5.0           # Overall heat loss coefficient [W/K]
)

(ρ = 880.0, C_p_s = 2000.0, C_p_l = 2200.0, Δh_m = 200000.0, T_m = 323.15, k_s = 0.24, k_l = 0.15, UA_loss = 5.0)

## Core Functions

In [2]:
"""
    PCMStorage

Struct holding PCM thermal storage parameters.
"""
struct PCMStorage
    V::Float64              # Volume [m³]
    m::Float64              # Mass [kg]
    ρ::Float64              # Density [kg/m³]
    C_p_s::Float64          # Specific heat solid [J/(kg·K)]
    C_p_l::Float64          # Specific heat liquid [J/(kg·K)]
    Δh_m::Float64           # Latent heat [J/kg]
    T_m::Float64            # Melting temperature [K]
    UA_loss::Float64        # Heat loss coefficient [W/K]
    E_max::Float64          # Maximum energy capacity [J]
    
    function PCMStorage(V::Float64;
            ρ::Float64 = 880.0,
            C_p_s::Float64 = 2.0e3,
            C_p_l::Float64 = 2.2e3,
            Δh_m::Float64 = 200.0e3,
            T_m::Float64 = 323.15,
            UA_loss::Float64 = 5.0,
            ΔT_sensible::Float64 = 10.0)  # Sensible temp range around T_m
        m = ρ * V
        # Max capacity: sensible (solid) + latent + sensible (liquid)
        E_max = m * (C_p_s * ΔT_sensible + Δh_m + C_p_l * ΔT_sensible)
        new(V, m, ρ, C_p_s, C_p_l, Δh_m, T_m, UA_loss, E_max)
    end
end

# Constructor with volume in liters
function PCMStorage(; V_liters::Float64, kwargs...)
    PCMStorage(V_liters / 1000.0; kwargs...)
end

PCMStorage

In [3]:
"""
    latent_capacity(pcm::PCMStorage) -> Float64

Calculate latent heat storage capacity [J].
"""
function latent_capacity(pcm::PCMStorage)
    return pcm.m * pcm.Δh_m
end

latent_capacity

In [4]:
"""
    energy_to_soc(pcm::PCMStorage, E::Float64) -> Float64

Convert stored energy [J] to state of charge [-].
"""
function energy_to_soc(pcm::PCMStorage, E::Float64)
    return clamp(E / pcm.E_max, 0.0, 1.0)
end

energy_to_soc

In [5]:
"""
    soc_to_energy(pcm::PCMStorage, SOC::Float64) -> Float64

Convert state of charge [-] to stored energy [J].
"""
function soc_to_energy(pcm::PCMStorage, SOC::Float64)
    return SOC * pcm.E_max
end

soc_to_energy

In [6]:
"""
    pcm_temperature(pcm::PCMStorage, SOC::Float64, ΔT_range::Float64=10.0) -> Float64

Estimate PCM temperature [K] from SOC.
Simplified model: T varies linearly in sensible regions, constant at T_m during phase change.
"""
function pcm_temperature(pcm::PCMStorage, SOC::Float64, ΔT_range::Float64=10.0)
    # Approximate boundaries
    E_sensible_s = pcm.m * pcm.C_p_s * ΔT_range
    E_latent = pcm.m * pcm.Δh_m
    
    SOC_melt_start = E_sensible_s / pcm.E_max
    SOC_melt_end = (E_sensible_s + E_latent) / pcm.E_max
    
    if SOC < SOC_melt_start
        # Solid sensible region
        return pcm.T_m - ΔT_range + (SOC / SOC_melt_start) * ΔT_range
    elseif SOC < SOC_melt_end
        # Phase change region (constant T)
        return pcm.T_m
    else
        # Liquid sensible region
        frac = (SOC - SOC_melt_end) / (1.0 - SOC_melt_end)
        return pcm.T_m + frac * ΔT_range
    end
end

pcm_temperature

In [7]:
"""
    heat_loss_rate(pcm::PCMStorage, T_pcm::Float64, T_amb::Float64) -> Float64

Calculate heat loss rate [W] to ambient.
"""
function heat_loss_rate(pcm::PCMStorage, T_pcm::Float64, T_amb::Float64)
    return pcm.UA_loss * max(T_pcm - T_amb, 0.0)
end

heat_loss_rate

In [8]:
"""
    update_pcm(pcm::PCMStorage, E_stored::Float64, Q_in::Float64, Q_out::Float64, 
               T_amb::Float64, Δt::Float64) -> NamedTuple

Update PCM state for one time step.
E_stored: current stored energy [J]
Q_in, Q_out: heat transfer rates [W]
T_amb: ambient temperature [K]
Δt: time step [s]
"""
function update_pcm(pcm::PCMStorage, E_stored::Float64, Q_in::Float64, Q_out::Float64, 
                    T_amb::Float64, Δt::Float64)
    # Current state
    SOC = energy_to_soc(pcm, E_stored)
    T_pcm = pcm_temperature(pcm, SOC)
    
    # Heat loss
    Q_loss = heat_loss_rate(pcm, T_pcm, T_amb)
    
    # Energy balance
    ΔE = (Q_in - Q_out - Q_loss) * Δt
    E_new = clamp(E_stored + ΔE, 0.0, pcm.E_max)
    SOC_new = energy_to_soc(pcm, E_new)
    T_pcm_new = pcm_temperature(pcm, SOC_new)
    
    return (
        E_stored = E_stored,
        E_new = E_new,
        SOC = SOC,
        SOC_new = SOC_new,
        T_pcm = T_pcm,
        T_pcm_new = T_pcm_new,
        Q_in = Q_in,
        Q_out = Q_out,
        Q_loss = Q_loss,
        ΔE = ΔE
    )
end

update_pcm

In [9]:
"""
    exergy_stored(pcm::PCMStorage, E_stored::Float64, T_0::Float64) -> Float64

Calculate exergy content [J] of stored thermal energy.
"""
function exergy_stored(pcm::PCMStorage, E_stored::Float64, T_0::Float64)
    SOC = energy_to_soc(pcm, E_stored)
    T_pcm = pcm_temperature(pcm, SOC)
    # Simplified: use average temperature for Carnot factor
    if T_pcm > T_0
        return E_stored * (1.0 - T_0 / T_pcm)
    else
        return 0.0
    end
end

exergy_stored

In [10]:
"""
    analyze_pcm(pcm::PCMStorage, E_stored::Float64, Q_in::Float64, Q_out::Float64,
                T_source::Float64, T_amb::Float64, T_0::Float64, Δt::Float64) -> NamedTuple

Complete energy and exergy analysis for one time step.
"""
function analyze_pcm(pcm::PCMStorage, E_stored::Float64, Q_in::Float64, Q_out::Float64,
                     T_source::Float64, T_amb::Float64, T_0::Float64, Δt::Float64)
    # Energy update
    state = update_pcm(pcm, E_stored, Q_in, Q_out, T_amb, Δt)
    
    # Exergy analysis
    Ex_stored_init = exergy_stored(pcm, E_stored, T_0)
    Ex_stored_final = exergy_stored(pcm, state.E_new, T_0)
    
    # Exergy of heat input (from source)
    Ex_in = Q_in * Δt * (1.0 - T_0 / T_source)
    
    # Exergy of heat output (useful)
    Ex_out = Q_out * Δt * (1.0 - T_0 / state.T_pcm)
    
    # Exergy of heat loss
    Ex_loss = state.Q_loss * Δt * (1.0 - T_0 / state.T_pcm)
    
    # Exergy destruction (from balance)
    Ex_d = Ex_in - (Ex_stored_final - Ex_stored_init) - Ex_out - Ex_loss
    Ex_d = max(Ex_d, 0.0)  # Numerical safety
    
    # Exergy efficiency (charging or discharging)
    if Q_in > Q_out
        # Charging: useful = stored exergy increase
        Ψ = Ex_in > 0 ? (Ex_stored_final - Ex_stored_init) / Ex_in : 0.0
    else
        # Discharging: useful = output exergy
        Ex_from_storage = Ex_stored_init - Ex_stored_final
        Ψ = Ex_from_storage > 0 ? Ex_out / Ex_from_storage : 0.0
    end
    
    return (
        # Energy quantities
        E_stored = E_stored,
        E_new = state.E_new,
        SOC = state.SOC,
        SOC_new = state.SOC_new,
        T_pcm = state.T_pcm,
        Q_in = Q_in,
        Q_out = Q_out,
        Q_loss = state.Q_loss,
        # Exergy quantities [J]
        Ex_stored = Ex_stored_final,
        Ex_in = Ex_in,
        Ex_out = Ex_out,
        Ex_loss = Ex_loss,
        Ex_d = Ex_d,
        Ψ = Ψ
    )
end

analyze_pcm

## Tests

In [11]:
using Test

function run_pcm_tests()
    println("Running PCM Storage Tests...")
    
    # Create test PCM: 100 liters
    pcm = PCMStorage(V_liters=100.0, T_m=323.15)
    
    # Test 1: Parameters
    @testset "Parameters" begin
        @test pcm.V ≈ 0.1 atol=1e-6  # 100 L = 0.1 m³
        @test pcm.m ≈ 0.1 * 880.0 atol=1e-6
        @test pcm.T_m == 323.15
    end
    
    # Test 2: Latent capacity
    @testset "Latent Capacity" begin
        E_latent = latent_capacity(pcm)
        @test E_latent ≈ pcm.m * pcm.Δh_m atol=1e-6
        @test E_latent > 0
    end
    
    # Test 3: SOC conversion
    @testset "SOC Conversion" begin
        @test energy_to_soc(pcm, 0.0) == 0.0
        @test energy_to_soc(pcm, pcm.E_max) == 1.0
        @test soc_to_energy(pcm, 0.5) ≈ 0.5 * pcm.E_max atol=1e-6
    end
    
    # Test 4: Temperature model
    @testset "Temperature Model" begin
        # At 50% SOC (phase change), T should be near T_m
        T_mid = pcm_temperature(pcm, 0.5)
        @test abs(T_mid - pcm.T_m) < 5.0  # Within 5K of melting point
    end
    
    # Test 5: Heat loss
    @testset "Heat Loss" begin
        Q_loss = heat_loss_rate(pcm, 323.15, 293.15)  # 30K difference
        @test Q_loss ≈ pcm.UA_loss * 30.0 atol=1e-6
        @test Q_loss > 0
    end
    
    # Test 6: Energy balance
    @testset "Energy Balance" begin
        E_init = 0.5 * pcm.E_max
        Q_in = 1000.0  # 1 kW charging
        Δt = 3600.0    # 1 hour
        
        state = update_pcm(pcm, E_init, Q_in, 0.0, 293.15, Δt)
        @test state.E_new > state.E_stored  # Energy increased
        @test state.SOC_new > state.SOC
    end
    
    # Test 7: Exergy analysis
    @testset "Exergy Analysis" begin
        E_init = 0.3 * pcm.E_max
        T_0 = 293.15
        T_source = 353.15  # 80°C source
        
        result = analyze_pcm(pcm, E_init, 2000.0, 0.0, T_source, 293.15, T_0, 3600.0)
        
        # Exergy in should be positive
        @test result.Ex_in > 0
        # Exergy stored should be positive
        @test result.Ex_stored > 0
        # Some exergy should be destroyed
        @test result.Ex_d >= 0
    end
    
    println("All tests passed!")
end

run_pcm_tests()

Running PCM Storage Tests...
Test Summary: | Pass  Total  Time
Parameters    |    3      3  0.0s
Test Summary:   | Pass  Total  Time
Latent Capacity |    2      2  0.0s
Test Summary:  | Pass  Total  Time
SOC Conversion |    3      3  0.0s
Test Summary:     | Pass  Total  Time
Temperature Model |    1      1  0.0s
Test Summary: | Pass  Total  Time
Heat Loss     |    2      2  0.0s
Test Summary:  | Pass  Total  Time
Energy Balance |    2      2  0.0s
Test Summary:   | Pass  Total  Time
Exergy Analysis |    3      3  0.0s
All tests passed!


## Example Usage

In [12]:
# Example: 200 liter PCM tank
pcm = PCMStorage(V_liters=200.0, T_m=323.15, Δh_m=200.0e3)

println("PCM Thermal Storage Specifications")
println("="^45)
println("Volume:             $(pcm.V * 1000) liters")
println("Mass:               $(round(pcm.m, digits=1)) kg")
println("Melting temp:       $(round(pcm.T_m - 273.15, digits=1)) °C")
println("Latent heat:        $(pcm.Δh_m/1000) kJ/kg")
println("─"^45)
println("Latent capacity:    $(round(latent_capacity(pcm)/3.6e6, digits=2)) kWh")
println("Total capacity:     $(round(pcm.E_max/3.6e6, digits=2)) kWh")
println("Heat loss coeff:    $(pcm.UA_loss) W/K")

PCM Thermal Storage Specifications
Volume:             200.0 liters
Mass:               176.0 kg
Melting temp:       50.0 °C
Latent heat:        200.0 kJ/kg
─────────────────────────────────────────────
Latent capacity:    9.78 kWh
Total capacity:     11.83 kWh
Heat loss coeff:    5.0 W/K


In [ ]:
# Simulate charging cycle (wrapped in let block for NBInclude compatibility)
let
    println("\nCharging Simulation (3 kW input)")
    println("="^55)
    println("Hour   SOC[%]   T_pcm[°C]   Q_loss[W]   Ex_stored[kJ]")
    println("─"^55)

    E = 0.1 * pcm.E_max  # Start at 10% SOC
    T_source = 353.15    # 80°C source
    T_amb = 293.15       # 20°C ambient
    T_0 = 293.15
    Δt = 3600.0          # 1 hour

    for hour in 0:6
        result = analyze_pcm(pcm, E, 3000.0, 0.0, T_source, T_amb, T_0, Δt)
        println("$(lpad(hour, 4))   $(lpad(round(result.SOC*100, digits=1), 6))    $(lpad(round(result.T_pcm - 273.15, digits=1), 8))     $(lpad(round(result.Q_loss, digits=0), 8))       $(lpad(round(result.Ex_stored/1000, digits=1), 10))")
        E = result.E_new
    end
end

## Export

Functions available for import:
- `PCMStorage` - Model struct
- `latent_capacity` - Latent heat storage capacity
- `energy_to_soc` / `soc_to_energy` - State of charge conversions
- `pcm_temperature` - Temperature from SOC
- `heat_loss_rate` - Heat loss to ambient
- `update_pcm` - Update state for time step
- `exergy_stored` - Exergy content
- `analyze_pcm` - Complete analysis